In [1]:
# Read bag file from record 
import pyrealsense2 as rs
import numpy as np
import cv2
import imutils
import os
import glob
import pandas as pd
from Process import LowVisionSubjectLocation
from Process import LowVisionSubjectAvoid


# file config
file_dir = ''
participant_num = 1
trial_type = 1 # 1: voice_intensity 2: voice_depth 3: brainport_intensity 4: brainport_depth, 5: Cane
filename_list = r"C:\Users\jinr2.STUDENT\data\20230307-155628.bag" #file name shoule be file_dir + trial_type + participant_num
participant_id = 'P1'

# find file at certain date
trial_date = "*0309*bag*" # ddmm
trial_yml = "*0309*yml*" 
trial_locate_c3d = 'LOCATE*c3d*'
trial_avoid_c3d = 'AVOID*c3d*'
trial_walk_c3d = 'WalK*c3d*'
folder_path = r'E:\data\test_results\P10\Session_preprocessed'

# Get a list of all files in the folder that match the pattern
file_list = glob.glob(os.path.join(folder_path, trial_date))

# # Print the list of matching files 
# for file_path in file_list:
#     print(file_path + "_" + str(os.path.getsize(file_path)/1024))

# find meta file data for further analysis 
file_list_yml = glob.glob(os.path.join(folder_path, trial_yml))

# for file_path in file_list_yml:
#     print(file_path + "_" + str(os.path.getsize(file_path)/1024))

# Get a list of WALK c3d trial 
file_list_locate_c3d = glob.glob(os.path.join(folder_path, trial_locate_c3d))
    
for file_path in file_list_locate_c3d:
    print(file_path + "_" + str(os.path.getsize(file_path)/1024))

# Get a list of AVOID c3d trial 
file_list_avoid_c3d = glob.glob(os.path.join(folder_path, trial_avoid_c3d))
    
for file_path in file_list_avoid_c3d:
    print(file_path + "_" + str(os.path.getsize(file_path)/1024))
    
# Get a list of LOCATE c3d trial  
file_list_walk_c3d = glob.glob(os.path.join(folder_path, trial_walk_c3d))
    
for file_path in file_list_walk_c3d:
    print(file_path + "_" + str(os.path.getsize(file_path)/1024))

[I 2023-11-23 17:34:16,939 WebStreaming] Starting Streaming Server...
[E 2023-11-23 17:34:17,136 OutputInterface] Serial port 'COM13' not available.


E:\data\test_results\P10\Session_preprocessed\LOCATE01.c3d_7207.84375
E:\data\test_results\P10\Session_preprocessed\LOCATE02.c3d_1745.0
E:\data\test_results\P10\Session_preprocessed\LOCATE03.c3d_1667.34375
E:\data\test_results\P10\Session_preprocessed\LOCATE04.c3d_2223.1875
E:\data\test_results\P10\Session_preprocessed\LOCATE05.c3d_12047.125
E:\data\test_results\P10\Session_preprocessed\LOCATE06.c3d_2907.734375
E:\data\test_results\P10\Session_preprocessed\LOCATE07.c3d_3593.40625
E:\data\test_results\P10\Session_preprocessed\LOCATE08.c3d_3134.0
E:\data\test_results\P10\Session_preprocessed\LOCATE09.c3d_1844.9375
E:\data\test_results\P10\Session_preprocessed\LOCATE10.c3d_2523.125
E:\data\test_results\P10\Session_preprocessed\LOCATE11.c3d_4732.125
E:\data\test_results\P10\Session_preprocessed\LOCATE12.c3d_2485.984375
E:\data\test_results\P10\Session_preprocessed\LOCATE13.c3d_2834.703125
E:\data\test_results\P10\Session_preprocessed\LOCATE14.c3d_5267.09375
E:\data\test_results\P10\Session

In [29]:
config_filepath = 'None' #file_list_yml[0]
# marker_filepath = r'E:\data\pilot_results\Low vision study_SS2303\Low Vision (1)\Pilot\Processed\locate01.c3d'
marker_filepath = r'E:\data\test_results\P7\Session_preprocessed\Locate14.c3d'
recording_filepath = 'None' #file_list[0]

self = LowVisionSubjectLocation(config_filepath, marker_filepath, recording_filepath)
self.load_marker(marker_filepath)
# self.load_config(config_filepath)
# subject.load_recording_bag(recording_filepath)

Evaluation C3Dserver
Version:  1.203
User:  Demonstration Copy
Organization:  Motion Lab Systems, Inc.


In [18]:
config_filepath = 'None' #file_list[0]
marker_filepath = r'E:\data\test_results\P2\Session_preprocessed\AVOID01.c3d'
recording_filepath = 'None' #file_list[0]

self = LowVisionSubjectAvoid(config_filepath, marker_filepath, recording_filepath)
self.load_marker(marker_filepath)
# self.load_config(config_filepath)
# subject.load_recording_bag(recording_filepath)

Evaluation C3Dserver
Version:  1.203
User:  Demonstration Copy
Organization:  Motion Lab Systems, Inc.


In [3]:
Participants_list = ["P10"]

for Participants_id in Participants_list:
    
    folder_path = os.path.join(r'E:\data\test_results', Participants_id ,'Session_preprocessed')
    file_list_locate_c3d = glob.glob(os.path.join(folder_path, 'LOCATE*c3d*'))
    
    results  = pd.DataFrame()

    for file_path in file_list_locate_c3d:

        config_filepath = 'None' 
        marker_filepath = file_path
        recording_filepath = "None" 

        self = LowVisionSubjectLocation(config_filepath, marker_filepath, recording_filepath)
        self.load_marker(marker_filepath)

        # Task 1 variable output
        vel_data, max_vel = self.velocity(self.get_marker('COFMAbsoluteVel'))
        hesitations, segments = self.hesitations(vel_data)
        max_cane, min_cane, avg_canevel = self.cane_parameters()

        # initialize data of lists.
        data = {'Response_time': [self.response_time()],\
                'Contact_time': [self.contact_time()],\
                'Average_velocity': [np.mean(vel_data)],\
                'Max_velocity': [max_vel],\
                 'Margin_stability': [self.margins_stability(int(self.response_time()*self.rate))],\
                'Margin_stability_all': [self.margins_stability_all()],\
                'Deviation' :[self.deviation()],\
                'Found_distance' : [self.found_distance()],\
               'Hesitation_number': [len(hesitations)],\
               'Travel_distance': [self.travel_distance(self.com[:,:])],\
               'Cane_left': [max_cane],\
               'Cane_right': [min_cane],\
               'Cane_avgvel': [avg_canevel],\
               'COFM': [self.get_marker('COFM')],\
               'COFM_absvel': [self.get_marker('COFMAbsoluteVel')],\
               'COFM_lvel': [self.get_marker('COFMLVel')],\
               'CYL_center':[self.get_marker('CYLcenter')],\
               'Canec':[self.get_marker('CANEc')],\
               'CaneAngle':[self.get_marker('CANEAngle')],\
               'Cane_vel': [self.get_marker('CANEsegAngVelDeg')],\
               'lheel' : [self.get_marker('LCAL')],\
                'llmal' : [self.get_marker('LLMAL')],\
                'lmmal' : [self.get_marker('LMMAL')],\
                'lmt5' : [self.get_marker('LMT5')],\
                'lmt23' : [self.get_marker('LMT23')],\
                'lmt1' : [self.get_marker('LMT1')],\
                'rheel' : [self.get_marker('RCAL')],\
                'rlmal' : [self.get_marker('RLMAL')],\
                'rmmal' : [self.get_marker('RMMAL')],\
                'rmt5' : [self.get_marker('RMT5')],\
                'rmt23' : [self.get_marker('RMT23')],\
                'rmt1' : [self.get_marker('RMT1')]}

        results = pd.concat([results, pd.DataFrame(data)], axis=0)

    results.to_json(os.path.join(folder_path,'results_locate.json'),orient="split")

Evaluation C3Dserver
Version:  1.203
User:  Demonstration Copy
Organization:  Motion Lab Systems, Inc.
Evaluation C3Dserver
Version:  1.203
User:  Demonstration Copy
Organization:  Motion Lab Systems, Inc.
Evaluation C3Dserver
Version:  1.203
User:  Demonstration Copy
Organization:  Motion Lab Systems, Inc.
Evaluation C3Dserver
Version:  1.203
User:  Demonstration Copy
Organization:  Motion Lab Systems, Inc.
Evaluation C3Dserver
Version:  1.203
User:  Demonstration Copy
Organization:  Motion Lab Systems, Inc.
Evaluation C3Dserver
Version:  1.203
User:  Demonstration Copy
Organization:  Motion Lab Systems, Inc.
Evaluation C3Dserver
Version:  1.203
User:  Demonstration Copy
Organization:  Motion Lab Systems, Inc.
Evaluation C3Dserver
Version:  1.203
User:  Demonstration Copy
Organization:  Motion Lab Systems, Inc.
Evaluation C3Dserver
Version:  1.203
User:  Demonstration Copy
Organization:  Motion Lab Systems, Inc.
Evaluation C3Dserver
Version:  1.203
User:  Demonstration Copy
Organizati

In [4]:
Participants_list = ['P10']

for Participants_id in Participants_list:
    
    folder_path = os.path.join(r'E:\data\test_results', Participants_id ,'Session_preprocessed')
    file_list_avoid_c3d = glob.glob(os.path.join(folder_path, 'AVOID*c3d*'))
        
    results  = pd.DataFrame()

    for file_path in file_list_avoid_c3d:

        config_filepath = 'None' 
        marker_filepath = file_path
        recording_filepath = "None" 

        self = LowVisionSubjectAvoid(config_filepath, marker_filepath, recording_filepath)
        self.load_marker(marker_filepath)

        # Task 2 variable output
        vel_data, max_vel = self.velocity(self.get_marker('COFMAbsoluteVel'))
        hesitations, segments = self.hesitations(vel_data)
        max_cane, min_cane, avg_canevel = self.cane_parameters()

        # initialize data of lists.
        data = {'Passing_time': [self.passing_time()],\
                'Average_velocity':[np.mean(vel_data)],\
                'Max_velocity': [max_vel],\
               'Hesitation_number': [len(hesitations)],\
               'Travel_distance': [self.travel_distance(self.com[:,:])],\
               'Cane_left': [max_cane],\
               'Cane_right': [min_cane],\
               'Cane_avgvel': [avg_canevel],\
               'Deviating_distance': [self.deviating_distance()],\
               'Clearing_distance': [self.clearing_distance()],\
                'found_times' : [self.dict_group['EVENT']],\
               'Margin_stability': [self.margins_stability(int(self.response_time*self.rate))],\
                'Margin_stability_all': [self.margins_stability_all()],\
               'COFM': [self.get_marker('COFM')],\
               'COFM_absvel': [self.get_marker('COFMAbsoluteVel')],\
               'COFM_lvel': [self.get_marker('COFMLVel')],\
               'CYL_center':[self.get_marker('CYLcenter')],\
               'small_square' : [self.get_marker('SMALLcenter')],\
               'large_square' : [self.get_marker('LARGEcenter')],\
               'Canec':[self.get_marker('CANEc')],\
               'CaneAngle':[self.get_marker('CANEAngle')],\
               'Cane_vel': [self.get_marker('CANEsegAngVelDeg')],\
               'lheel' : [self.get_marker('LCAL')],\
                'llmal' : [self.get_marker('LLMAL')],\
                'lmmal' : [self.get_marker('LMMAL')],\
                'lmt5' : [self.get_marker('LMT5')],\
                'lmt23' : [self.get_marker('LMT23')],\
                'lmt1' : [self.get_marker('LMT1')],\
                'rheel' : [self.get_marker('RCAL')],\
                'rlmal' : [self.get_marker('RLMAL')],\
                'rmmal' : [self.get_marker('RMMAL')],\
                'rmt5' : [self.get_marker('RMT5')],\
                'rmt23' : [self.get_marker('RMT23')],\
                'rmt1' : [self.get_marker('RMT1')]}

        results = pd.concat([results, pd.DataFrame(data)], axis=0)

    results.to_json(os.path.join(folder_path,'results_avoid.json'),orient="split")

Evaluation C3Dserver
Version:  1.203
User:  Demonstration Copy
Organization:  Motion Lab Systems, Inc.
Evaluation C3Dserver
Version:  1.203
User:  Demonstration Copy
Organization:  Motion Lab Systems, Inc.
Evaluation C3Dserver
Version:  1.203
User:  Demonstration Copy
Organization:  Motion Lab Systems, Inc.
Evaluation C3Dserver
Version:  1.203
User:  Demonstration Copy
Organization:  Motion Lab Systems, Inc.
Evaluation C3Dserver
Version:  1.203
User:  Demonstration Copy
Organization:  Motion Lab Systems, Inc.
Evaluation C3Dserver
Version:  1.203
User:  Demonstration Copy
Organization:  Motion Lab Systems, Inc.
Evaluation C3Dserver
Version:  1.203
User:  Demonstration Copy
Organization:  Motion Lab Systems, Inc.
Evaluation C3Dserver
Version:  1.203
User:  Demonstration Copy
Organization:  Motion Lab Systems, Inc.
Evaluation C3Dserver
Version:  1.203
User:  Demonstration Copy
Organization:  Motion Lab Systems, Inc.
Evaluation C3Dserver
Version:  1.203
User:  Demonstration Copy
Organizati

In [5]:
Participants_list = ['P10'] #,'P2','P3','P5','P7']

for Participants_id in Participants_list:
    
    folder_path = os.path.join(r'E:\data\test_results', Participants_id ,'Session_preprocessed')
    file_list_walk_c3d = glob.glob(os.path.join(folder_path, 'WalK*c3d*'))

    results  = pd.DataFrame()

    for file_path in file_list_walk_c3d:

        config_filepath = 'None' 
        marker_filepath = file_path
        recording_filepath = "None" 

        self = LowVisionSubjectLocation(config_filepath, marker_filepath, recording_filepath)
        self.load_marker(marker_filepath)

        # Task 1 variable output
        vel_data, max_vel = self.velocity(self.get_marker('COFMAbsoluteVel'))
        max_cane, min_cane, avg_canevel = self.cane_parameters()

        # initialize data of lists.
        data = {'Response_time': [self.response_time()],\
                'Contact_time': [self.contact_time()],\
                'Average_velocity': [np.mean(vel_data)],\
                'Max_velocity': [max_vel],\
                 'Margin_stability': [self.margins_stability(int(self.response_time()*self.rate))],\
                'Margin_stability_all': [self.margins_stability_all()],\
               'Travel_distance': [self.travel_distance(self.com[:,:])],\
               'Cane_left': [max_cane],\
               'Cane_right': [min_cane],\
               'Cane_avgvel': [avg_canevel],\
               'COFM': [self.get_marker('COFM')],\
               'COFM_absvel': [self.get_marker('COFMAbsoluteVel')],\
               'COFM_lvel': [self.get_marker('COFMLVel')],\
               'Canec':[self.get_marker('CANEc')],\
               'CaneAngle':[self.get_marker('CANEAngle')],\
               'Cane_vel': [self.get_marker('CANEsegAngVelDeg')],\
               'lheel' : [self.get_marker('LCAL')],\
                'llmal' : [self.get_marker('LLMAL')],\
                'lmmal' : [self.get_marker('LMMAL')],\
                'lmt5' : [self.get_marker('LMT5')],\
                'lmt23' : [self.get_marker('LMT23')],\
                'lmt1' : [self.get_marker('LMT1')],\
                'rheel' : [self.get_marker('RCAL')],\
                'rlmal' : [self.get_marker('RLMAL')],\
                'rmmal' : [self.get_marker('RMMAL')],\
                'rmt5' : [self.get_marker('RMT5')],\
                'rmt23' : [self.get_marker('RMT23')],\
                'rmt1' : [self.get_marker('RMT1')]}

        results = pd.concat([results, pd.DataFrame(data)], axis=0)

    results.to_json(os.path.join(folder_path,'results_walk.json'),orient="split")

Evaluation C3Dserver
Version:  1.203
User:  Demonstration Copy
Organization:  Motion Lab Systems, Inc.
Evaluation C3Dserver
Version:  1.203
User:  Demonstration Copy
Organization:  Motion Lab Systems, Inc.
